In [1]:
import mlflow
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from datetime import datetime

from mlflow.tracking import MlflowClient
from mlflow.utils.rest_utils import http_request
import uuid

subscription_id = '96aede12-2f73-41cb-b983-6d11a904839b'
resource_group = 'promptflow'
workspace_name = 'promptflow-canary-dev'

# subscription_id = '96aede12-2f73-41cb-b983-6d11a904839b'
# resource_group = 'hod-rg'
# workspace_name = 'hod-pflow'

ml_client = MLClient(credential=DefaultAzureCredential(),
                        subscription_id=subscription_id,
                        resource_group_name=resource_group,
                        workspace_name=workspace_name)

mlflow_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
print(mlflow_tracking_uri)
mlflow.set_tracking_uri(mlflow_tracking_uri)

# update the cred host for api call
client = MlflowClient()
cred = client._tracking_client.store.get_host_creds()
# cred.host = cred.host.replace("mlflow/v2.0", "mlflow/v1.0").replace("mlflow/v1.0", "history/v1.0")
cred.host = cred.host.replace("mlflow/v2.0", "mlflow/v1.0").replace("mlflow/v1.0", "index/v1.0")
print(cred.host)

azureml://eastus2euap.api.azureml.ms/mlflow/v1.0/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/promptflow/providers/Microsoft.MachineLearningServices/workspaces/promptflow-canary-dev
https://eastus2euap.api.azureml.ms/index/v1.0/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/promptflow/providers/Microsoft.MachineLearningServices/workspaces/promptflow-canary-dev


In [ ]:
credential=DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default").token

In [28]:
runs = mlflow.search_runs(max_results=50, output_format="list", order_by=["start_time DESC"])

In [30]:
for run in runs[0:10]:
    # print(f"{run.info.run_id} {run.info.status} {run.info.start_time} {run.info.end_time}")
    print(run.to_dictionary())

{'info': {'artifact_uri': '', 'end_time': 1688532651851, 'experiment_id': '108bbbb1-af43-4246-96ea-782056d4e7fd', 'lifecycle_stage': 'active', 'run_id': 'dd28ab79-99cd-4bc2-b6b6-b51bfc9c4b58', 'run_name': 'quirky_juice_h86hyrnh', 'run_uuid': 'dd28ab79-99cd-4bc2-b6b6-b51bfc9c4b58', 'start_time': 1688532651564, 'status': 'FINISHED', 'user_id': '1e6ca164-23af-452c-8b54-192295bb78ef'}, 'data': {'metrics': {}, 'params': {}, 'tags': {'mlflow.rootRunId': 'dd28ab79-99cd-4bc2-b6b6-b51bfc9c4b58', 'mlflow.runName': 'quirky_juice_h86hyrnh', 'mlflow.user': '1e6ca164-23af-452c-8b54-192295bb78ef'}}, 'inputs': {'dataset_inputs': []}}
{'info': {'artifact_uri': '', 'end_time': 1688532650797, 'experiment_id': '108bbbb1-af43-4246-96ea-782056d4e7fd', 'lifecycle_stage': 'active', 'run_id': 'classification_eval_flow_run_id_6dc468bb-27a4-481a-bf82-14767714e5ec', 'run_name': 'great_hominy_93rmvbp1', 'run_uuid': 'classification_eval_flow_run_id_6dc468bb-27a4-481a-bf82-14767714e5ec', 'start_time': 1688532648171,

In [ ]:
a = ml_client.jobs._runs_operations.get_run(run_id="f9f19743-f9bd-4046-9074-35241f9286a9")
print(a)

In [ ]:
run = mlflow.get_run(run_id="run_20230524162058_3b2d8cb0-1b57-498e-8f1a-aff518897bb0_variant_2")
print(run.to_dictionary())

In [ ]:
run = mlflow.active_run()
print(run.info.run_id)

In [ ]:

from enum import Enum
class Status(Enum):
    Started = "Started"
    Completed = "Completed"
    Failed = "Failed"
    Cancelled = "Cancelled"
    NOTSTARTED = 'NotStarted'

    @staticmethod
    def is_terminated(status):
        return status in {Status.Completed, Status.Failed, Status.Cancelled}
    
    
print(Status.is_terminated(Status.Completed))

In [ ]:
class AzureMLConfig:
    SUBSCRIPTION_ID = "SUBSCRIPTION_ID"
    RESOURCE_GROUP_NAME = "RESOURCE_GROUP_NAME"
    WORKSPACE_NAME = "WORKSPACE_NAME"
    WORKSPACE_REGION = "WORKSPACE_REGION"
    
    @staticmethod
    def test():
        print("test")

class hod(AzureMLConfig):
    a = 1

hod.test()

## Create a run with NotStarted status

In [2]:
run_id = str(uuid.uuid4())
print(f"New run id: {run_id}")

response = http_request(
    host_creds=cred,
    endpoint="/experiments/{}/runs/{}".format("promptflow", run_id),
    method="PATCH",
    json={"runId": run_id},
)
if response.status_code == 200:
    print(f"Successfully created a run with run id '{run_id}'")
else:
    raise


New run id: f172595d-9b91-46b4-90cf-0fca214a6907
Successfully created a run with run id 'f172595d-9b91-46b4-90cf-0fca214a6907'


## Update run status with API call

In [5]:
run_id = "f172595d-9b91-46b4-90cf-0fca214a6907"
mlflow_run = mlflow.get_run(run_id=run_id)
experiment_id = mlflow_run.info.experiment_id

status_event = {
    "timestamp": datetime.utcnow().isoformat(),
    "name": "Microsoft.MachineLearning.Run.Start",
    "data": {
        "startTime": datetime.utcnow().isoformat(),
    },
}

response = http_request(
    host_creds=cred,
    endpoint="/experimentids/{}/runs/{}/events".format(experiment_id, run_id),
    method="POST",
    json=status_event,
)

if response.status_code == 200:
    print(f"Successfully update run status to 'Running' for run {run_id!r}.")
else:
    print(
        f"Failed to update run status to 'Running' for run {run_id!r}. "
        f"Code: {response.status_code}, text: {response.text}"
    )

Successfully update run status to 'Running' for run 'f172595d-9b91-46b4-90cf-0fca214a6907'.


## Cancel an active run

In [ ]:

current_run = mlflow.active_run()
run_id = current_run.info.run_id
experiment_id = current_run.info.experiment_id

end_event = {
    "timestamp": datetime.utcnow().isoformat(),
    "name": "Microsoft.MachineLearning.Run.Canceled",
    "data": {
        "endtime": datetime.utcnow().isoformat(),
    },
}
response = http_request(
    host_creds=cred,
    endpoint="/experimentids/{}/runs/{}/events".format(experiment_id, run_id),
    method="POST",
    json=end_event,
)

if response.status_code == 200:
    print(f"Successfully canceled a run with run id '{run_id}'")
else:
    print(f"Code: {response.status_code}, text: {response.text}")

## Write error message

In [17]:
from datetime import datetime
# run = mlflow.start_run()
# print(run.info.run_id)

# current_run = mlflow.active_run()
# run_id = current_run.info.run_id
# experiment_id = current_run.info.experiment_id

error_event = {
    "timestamp": datetime.utcnow().isoformat(),
    "name": "Microsoft.MachineLearning.Run.Error",
    "data": {
        "errorResponse":{
            "error": {
                "code": "UserError",
                "message": "test hod error",
            },
            "componentName": "promptflow"
        },
    },
}
response = http_request(
    host_creds=cred,
    endpoint="/experimentids/{}/runs/{}/events".format(experiment_id, run_id),
    method="POST",
    json=error_event,
)

if response.status_code == 200:
    print(f"Successfully write error message with run id '{run_id}'")
else:
    print(f"Code: {response.status_code}, text: {response.text}")

Successfully write error message with run id '1ec6cf78-8583-43cf-a933-3b0c8b5fbedf'


## Write run properties

In [ ]:
run = mlflow.start_run()
print(run.info.run_id)

current_run = mlflow.active_run()
run_id = current_run.info.run_id
experiment_id = current_run.info.experiment_id

properties = {
    "hod-test-property": "123"
}

response = http_request(
    host_creds=cred,
    endpoint="/experimentids/{}/runs/{}".format(experiment_id, run_id),
    method="PATCH",
    json={"runId": run_id, "properties": properties},
)

if response.status_code == 200:
    print(f"Successfully write run properties with run id '{run_id}'")
else:
    print(f"Code: {response.status_code}, text: {response.text}")

## List runs from index service

In [2]:
import importlib  
importlib.reload(mlflow) 

pay_load = {
	"filters": [{
			"field": "type",
			"operator": "eq",
			"values": ["runs"]
		}, {
			"field": "annotations/archived",
			"operator": "eq",
			"values": ["true"]
		},
        {
			"field": "properties/runType",
            "operator": "contains",
            "values": ["azureml.promptflow.FlowRun", "azureml.promptflow.EvaluationRun"]
		}
	],
	"freeTextSearch": "",
	"order": [{
			"direction": "Desc",
			"field": "properties/creationContext/createdTime"
		}
	],
	"pageSize": 50,
	"skip": 0,
	"includeTotalResultCount": True,
	"searchBuilder": "AppendPrefix"
}

response = http_request(
    host_creds=cred,
    endpoint="/entities",
    method="POST",
    json=pay_load,
)

if response.status_code == 200:
    print(f"Successfully list runs from index service")
else:
    print(f"Code: {response.status_code}, text: {response.text}")

{'User-Agent': 'mlflow-python-client/2.4.1 azureml-mlflow/1.50.0'}
Successfully list runs from index service


In [73]:
import json
res = json.loads(response.text)

print(f"totalCount: {res['totalCount']}")
print(f"Length of value: {len(res['value'])}")
print(f"nextSkip: {res['nextSkip']}")

with open("./entities_archived.json", "w") as f:
    json.dump(res, f)


totalCount: 0
Length of value: 0
nextSkip: None


In [ ]:
credential=DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default").token
print(token)

In [6]:
import urllib3
import json

http = urllib3.PoolManager() 
url = "https://eastus2euap.api.azureml.ms/index/v1.0/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/promptflow/providers/Microsoft.MachineLearningServices/workspaces/promptflow-canary-dev/entities"

payload = {
	"filters": [{
			"field": "type",
			"operator": "eq",
			"values": ["runs"]
		}, {
			"field": "annotations/archived",
			"operator": "eq",
			"values": ["false"]
		},
        {
			"field": "properties/runType",
            "operator": "contains",
            "values": ["azureml.promptflow.FlowRun", "azureml.promptflow.EvaluationRun"]
		}
	],
	"freeTextSearch": "",
	"order": [{
			"direction": "Desc",
			"field": "properties/creationContext/createdTime"
		}
	],
	"pageSize": 50,
	"skip": 0,
	"includeTotalResultCount": True,
	"searchBuilder": "AppendPrefix"
}
payload = json.dumps(payload).encode('utf-8')

headers = {  
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json',
} 

response = http.request("POST", url, body=payload, headers=headers)

if response.status == 200:
    print(f"Successfully list runs from index service")
else:
    print(f"Code: {response.status}, Reason: {response.reason}")

Successfully list runs from index service


In [8]:
# print(response.data.decode('utf-8'))
import json
res = json.loads(response.data)

print(f"totalCount: {res['totalCount']}")
print(f"Length of value: {len(res['value'])}")
print(f"nextSkip: {res['nextSkip']}")

with open("./entities_archived.json", "w") as f:
    json.dump(res, f)

totalCount: 26341
Length of value: 50
nextSkip: 50


In [62]:
a = [1,2,3]
print(a[0:0])

[]


In [70]:
print(ml_client.registry_name)

AttributeError: 'MLClient' object has no attribute 'registry_name'